In [ ]:
import functools
import os
from pathlib import Path
import sys

from dotenv import load_dotenv
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import scipy.interpolate as interpolate

sys.path.insert(0, '..')
from LEAP3D.config import DATA_DIR
from LEAP3D.scanning import ScanParameters
from LEAP3D.scanning import ScanResults

In [ ]:
ANIMATION_FRAME_DURATION = 1
GIF_FPS = 60
PLOT_DIR = Path("../Plots/")

In [ ]:
case_index = 0
case_filename = DATA_DIR / "case_0000.npz"
params_filename =  DATA_DIR / "Params.npy"
rough_coord_filename =  DATA_DIR / "Rough_coord.npz"

In [ ]:
X_MIN, X_MAX = -0.0012, 0.0012
Y_MIN, Y_MAX = -0.0012, 0.0012
Z_MIN, Z_MAX = -0.0006, 0
SAFETY_OFFSET = 0.0005
MELTING_POINT = 1723

In [ ]:
case_params = ScanParameters(params_filename, case_index,
                             x_min=X_MIN, x_max=X_MAX,
                             y_min=Y_MIN, y_max=Y_MAX,
                             z_min=Z_MIN, z_max=Z_MAX,
                             safety_offset=SAFETY_OFFSET,
                             melting_point=MELTING_POINT)
case_params

In [ ]:
case_results = ScanResults(case_filename, rough_coord_filename)

In [ ]:
def animate_top_heatmap(case_results, case_params, frames=None, show_only_melt=False):
    def animate(i, ax, case_results, melting_point, show_only_melt):
        X, Y, temperature = case_results.get_top_layer_temperatures(i)

        if show_only_melt:
            temperature[temperature < melting_point] = 0
            temperature[temperature >= melting_point] = 1

        im = ax.pcolormesh(X, Y, temperature, animated=True, vmax=1 if show_only_melt else melting_point)
        return im,

    x_min, x_max, y_min, y_max, *_ = case_params.get_bounds()
    melting_point = case_params.melting_point
    safety_offset = case_params.safety_offset

    fig, ax = plt.subplots(sharex=True, sharey=True)
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)

    frames = frames if frames is not None else case_results.total_timesteps
    ims = [animate(i, ax, case_results, melting_point, show_only_melt) for i in range(0, frames, 5)]

    laser_x_min = x_min + safety_offset
    laser_x_max = x_max - safety_offset
    laser_y_min = y_min + safety_offset
    laser_y_max = y_max - safety_offset
    ax.plot([laser_x_min, laser_x_min], [laser_y_min, laser_y_max], 'black', lw=1)
    ax.plot([laser_x_max, laser_x_max], [laser_y_min, laser_y_max], 'black', lw=1)
    ax.plot([laser_x_min, laser_x_max], [laser_y_min, laser_y_min], 'black', lw=1)
    ax.plot([laser_x_min, laser_x_max], [laser_y_max, laser_y_max], 'black', lw=1)

    if not show_only_melt:
        fig.colorbar(ims[0][0], ax=ax)

    return fig, ims

In [ ]:
fig, ims = animate_top_heatmap(case_results, case_params)
fig.get_axes()[0].set_title("Top layer view of temperature map over time")
ani = animation.ArtistAnimation(fig, ims, interval=ANIMATION_FRAME_DURATION, blit=True,
                                repeat_delay=1000)
ani.save(PLOT_DIR / "top_heatmap.gif", fps=GIF_FPS)

In [ ]:
fig, ims = animate_top_heatmap(case_results, case_params, show_only_melt=True)
fig.get_axes()[0].set_title("Top layer view of melt pool over time")
ani = animation.ArtistAnimation(fig, ims, interval=ANIMATION_FRAME_DURATION, blit=True,
                                repeat_delay=1000)
ani.save(PLOT_DIR / "top_melt.gif", fps=GIF_FPS)

In [ ]:
def get_interpolated_data_at_timestep(
        coordinates, temperature, laser_angle, laser_position,
        x_min=None, x_max=None, y_min=None, y_max=None, z_min=None, z_max=None,
        steps=128, method='nearest', return_grid=False
        ):
    x_min = x_min if x_min is not None else min(coordinates, key=lambda point: point[0])[0]
    x_max = x_max if x_max is not None else max(coordinates, key=lambda point: point[0])[0]
    y_min = y_min if y_min is not None else min(coordinates, key=lambda point: point[1])[1]
    y_max = y_max if y_max is not None else max(coordinates, key=lambda point: point[1])[1]
    z_min = z_min if z_min is not None else min(coordinates, key=lambda point: point[2])[2]
    z_max = z_max if z_max is not None else max(coordinates, key=lambda point: point[2])[2]

    laser_x, laser_y, _ = laser_position
    laser_angle_tan = np.tan(laser_angle)
    get_y = lambda x: laser_angle_tan * (x - laser_x) + laser_y

    xi = np.linspace(x_min, x_max, steps)
    zi = np.linspace(z_min, z_max, 64)
    yi = get_y(xi)
    yi_within_bounds_indices = np.where((yi >= y_min) & (yi <= y_max))  # Keep elements within bounds
    yi = yi[yi_within_bounds_indices]
    xi = xi[yi_within_bounds_indices]

    cross_section_points = np.array([(x, y, z) for (x, y) in zip(xi, yi) for z in zi])

    D_interpolated = interpolate.griddata(
        coordinates,
        temperature,
        cross_section_points,
        method=method)

    if return_grid:
        return D_interpolated, xi, yi, zi
    return D_interpolated

In [ ]:
def animate_temperature_cross_section(case_results, case_params, frames=None, show_only_melt=False):
    def project_point(x, y, x_min, y_min):
        return np.sqrt((x - x_min)**2 + (y - y_min)**2) - np.sqrt(x_min**2 + y_min**2)

    def animate(i, case_results, case_params, show_only_melt):
        print(i, end='\r')
        # TODO: add laser position
        temperature = case_results.get_rough_temperatures_at_timestep(i).flatten()
        melt_pool_coordinates, melt_pool_temperature = case_results.get_melt_pool_coordinates_and_temperature(i)

        all_coordinates = rough_coords + melt_pool_coordinates
        all_temperatures = list(temperature) + melt_pool_temperature

        laser_position = case_results.get_laser_coordinates_at_timestep(i)

        x_min, x_max, y_min, y_max, z_min, z_max = case_params.get_bounds()
        melting_point = case_params.melting_point
        safety_offset = case_params.safety_offset

        # Since the rough resolution is at most 64x64, for the cross section
        # we can interpolate for around sqrt(64**2 + 64**2) approx. 96 steps.
        steps = 96
        T_interpolated, xi, yi, zi = get_interpolated_data_at_timestep(
            all_coordinates, all_temperatures,
            laser_angle, laser_position,
            x_min, x_max, y_min, y_max, z_min, z_max,
            return_grid=True, steps=steps, method='nearest')

        if show_only_melt:
            T_interpolated[T_interpolated < melting_point] = 0
            T_interpolated[T_interpolated >= melting_point] = 1

        # Project x and y axes into 1D
        new_x = project_point(xi, yi, x_min, y_min)

        # Reshape coordinates for plotting
        new_x_grid = [[x] * zi.shape[0] for x in new_x]
        new_z_grid = [zi for _ in new_x]
        T_values = T_interpolated.reshape((new_x.shape[0], zi.shape[0]))

        im = ax.pcolormesh(new_x_grid, new_z_grid, T_values, animated=True, vmax=1 if show_only_melt else melting_point)

        # Calculate scanning bounds of the cross section
        laser_y_min = y_min + safety_offset
        laser_x_min = (laser_y_min - laser_position[1]) / laser_angle_tan + laser_position[0]
        if laser_x_min < x_min + safety_offset:
            laser_x_min = x_min + safety_offset
            laser_y_min = laser_angle_tan * (laser_x_min - laser_position[0]) + laser_position[1]

        laser_y_max = y_max - safety_offset
        laser_x_max = (laser_y_max - laser_position[1]) / laser_angle_tan + laser_position[0]
        if laser_x_max > x_max - safety_offset:
            laser_x_max = x_max - safety_offset
            laser_y_max = laser_angle_tan * (laser_x_max - laser_position[0]) + laser_position[1]

        # Project the scanning bounds into the cross section
        cs_x_min = project_point(laser_x_min, laser_y_min, x_min, y_min)
        cs_x_max = project_point(laser_x_max, laser_y_max, x_min, y_min)

        # Plot the scanning bounds
        scanning_bound_left, = ax.plot([cs_x_min, cs_x_min], [z_min, z_max], 'black', lw=1)
        scanning_bound_right, = ax.plot([cs_x_max, cs_x_max], [z_min, z_max], 'black', lw=1)

        return [im, scanning_bound_left, scanning_bound_right]


    rough_coords = case_results.get_rough_coordinates_for_cross_section()

    laser_angle = case_params.scanning_angle
    laser_angle_tan = np.tan(laser_angle)

    fig, ax = plt.subplots(sharex=True, sharey=True)
    ax.set_xlim(
        -np.sqrt(case_params.x_min**2 + case_params.y_min**2),
        np.sqrt(case_params.x_max**2 + case_params.y_max**2))

    frames = frames if frames is not None else case_results.total_timesteps
    animate_partial = functools.partial(animate,
                                        case_results=case_results,
                                        case_params=case_params,
                                        show_only_melt=show_only_melt)
    ims = [animate_partial(i) for i in range(0, frames, 5)]

    if not show_only_melt:
        fig.colorbar(ims[0][0], ax=ax)

    return fig, ims

In [ ]:
fig, ims = animate_temperature_cross_section(case_results, case_params)

In [ ]:
fig.get_axes()[0].set_title("Cross section of temperature map over time")
ani = animation.ArtistAnimation(fig, ims, repeat=True, interval=ANIMATION_FRAME_DURATION, blit=True, repeat_delay=1000)
ani.save(PLOT_DIR / "temperature_cross_section.gif", fps=GIF_FPS, progress_callback=print)


In [ ]:
fig, ims = animate_temperature_cross_section(case_results, case_params, show_only_melt=True)

In [ ]:
fig.get_axes()[0].set_title("Cross section of melt pool over time")
ani = animation.ArtistAnimation(fig, ims, repeat=True, interval=ANIMATION_FRAME_DURATION, blit=True, repeat_delay=1000)
ani.save(PLOT_DIR / "melt_cross_section.gif", fps=GIF_FPS, progress_callback=print)

In [ ]:
def animate_laser_trajectory_2d(case_results, case_params, frames=None):
    def animate(i, ax, case_results):
        laser_x, laser_y, _ = case_results.get_laser_coordinates_at_timestep(i)
        image = ax.scatter(laser_x, laser_y, animated=True, c='red')
        return image,

    x_min, x_max, y_min, y_max, *_ = case_params.get_bounds()
    safety_offset = case_params.safety_offset

    fig = plt.figure()
    ax = fig.add_subplot()
    ax.set_aspect('equal', adjustable='box')
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)

    # TODO: Refactor?
    laser_x_min = x_min + safety_offset
    laser_x_max = x_max - safety_offset
    laser_y_min = y_min + safety_offset
    laser_y_max = y_max - safety_offset
    ax.plot([laser_x_min, laser_x_min], [laser_y_min, laser_y_max], 'black', lw=1)
    ax.plot([laser_x_max, laser_x_max], [laser_y_min, laser_y_max], 'black', lw=1)
    ax.plot([laser_x_min, laser_x_max], [laser_y_min, laser_y_min], 'black', lw=1)
    ax.plot([laser_x_min, laser_x_max], [laser_y_max, laser_y_max], 'black', lw=1)

    frames = frames if frames is not None else case_results.total_timesteps
    return fig, [animate(i, ax, case_results) for i in range(0, frames, 5)]


In [ ]:
fig, ims = animate_laser_trajectory_2d(case_results, case_params)
fig.get_axes()[0].set_title("Top view of laser position over time")
ani = animation.ArtistAnimation(fig, ims, repeat=True, interval=ANIMATION_FRAME_DURATION, blit=True, repeat_delay=1000)
ani.save(PLOT_DIR / "laser_position.gif", fps=GIF_FPS, progress_callback=print)